In [1]:
import preprocess.pp_v4 as pp
from preprocess.pp_basic import BASE_DIR, RAW_DIR, DATA_DIR, docs

/Users/won/dev/00_codeit/0_mission/200_DL_RAG/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import re
import os
import json
from pathlib import Path
import pdfplumber

In [3]:
PDF_PATH = str(docs[0])

## 표 데이터 추출 이쁘게

In [4]:
import pandas as pd

# import docling.utils.model_downloader
# docling.utils.model_downloader.download_models()
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import EasyOcrOptions, PdfPipelineOptions, TableFormerMode

pipeline_options = PdfPipelineOptions(do_table_structure=True)
pipeline_options.table_structure_options.mode = TableFormerMode.ACCURATE  # use more accurate TableFormer model
pipeline_options.table_structure_options.do_cell_matching = True  # uses text cells predicted from table structure model

doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

doc = doc_converter.convert(PDF_PATH).document

In [5]:
new_dict = dict()
for tuple_ in list(doc):
    new_dict[tuple_[0]] = tuple_[1]

In [6]:
import json
import re
from collections import Counter, defaultdict

# =============================================================================
# 헬퍼 함수들
# =============================================================================

def resolve_ref(doc, ref_string):
    """RefItem을 실제 객체로 변환"""
    if not ref_string or not ref_string.startswith('#/'):
        return None
    
    parts = ref_string.split('/')
    if len(parts) < 3:
        return None
    
    collection = parts[1]
    idx = int(parts[2])
    
    try:
        if collection == 'texts':
            return doc.texts[idx]
        elif collection == 'groups':
            return doc.groups[idx]
        elif collection == 'tables':
            return doc.tables[idx]
        elif collection == 'pictures':
            return doc.pictures[idx]
    except (IndexError, AttributeError):
        return None
    
    return None

def safe_export_markdown(table):
    """테이블을 마크다운으로 안전하게 변환"""
    try:
        if hasattr(table, 'export_to_markdown'):
            result = table.export_to_markdown()
            if isinstance(result, str):
                return result
    except Exception as e:
        print(f"Warning: 테이블 마크다운 변환 실패 - {e}")
    return None

# =============================================================================
# 노이즈 제거 클래스
# =============================================================================

class SearchIndexCleaner:
    """검색 인덱스 노이즈 제거"""
    
    def __init__(self):
        self.exclude_labels = [
            'page_header',
            'page_footer', 
            'checkbox_unselected',
            'checkbox_selected'
        ]
        
        self.noise_patterns = [
            r'^\d+\.$',                    # "1.", "2."
            r'^\d+$',                       # "1", "2"
            r'^[A-Za-z]$',                  # 단일 문자
            r'^[□○●◆◇▪▫⚬◦*-]+$',         # 기호만
            r'^M[+-]?\d*$',                 # "M", "M-1"
            r'^제\s*\d+\s*조$',             # "제1조"
            r'^[가-힣]{1,2}$',               # 1-2자 한글
        ]
        
        self.valid_short_texts = {
            '목차', '서론', '결론', '요약', '개요',
            '배경', '목적', '방법', '결과', '고찰'
        }
    
    def is_noise(self, text, label=None):
        """노이즈 여부 판단"""
        text = text.strip()
        
        # 화이트리스트
        if text in self.valid_short_texts:
            return False
        
        # 섹션 헤더/캡션은 관대
        if label in ['section_header', 'caption'] and len(text) >= 2:
            return False
        
        # 제외 라벨
        if label in self.exclude_labels:
            return True
        
        # 길이 제한
        if len(text) <= 2:
            return True
        
        # 노이즈 패턴
        for pattern in self.noise_patterns:
            if re.match(pattern, text):
                return True
        
        # 반복 문자
        words = text.split()
        if len(words) > 3:
            word_counts = Counter(words)
            if any(count >= 3 for count in word_counts.values()):
                return True
        
        # 특수문자 비율
        special_chars = sum(1 for c in text if not c.isalnum() and not c.isspace())
        if len(text) > 0 and special_chars / len(text) > 0.5:
            return True
        
        # 짧은 숫자만
        if text.replace(' ', '').replace('.', '').replace(',', '').isdigit():
            if len(text.replace(' ', '').replace('.', '').replace(',', '')) < 4:
                return True
        
        return False
    
    def normalize_text(self, text):
        """텍스트 정규화"""
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\.{4,}', '...', text)
        text = re.sub(r'·{3,}', '...', text)
        return text.strip()
    
    def clean_search_index(self, search_index):
        """검색 인덱스 정제"""
        cleaned = []
        
        for item in search_index:
            content = item['content']
            label = item['metadata'].get('label')
            
            if self.is_noise(content, label):
                continue
            
            content = self.normalize_text(content)
            
            if len(content.strip()) < 3 and label != 'section_header':
                continue
            
            item['content'] = content
            cleaned.append(item)
        
        return cleaned

# =============================================================================
# 계층 추론 클래스
# =============================================================================

class HierarchyDetector:
    """문서 계층 자동 추론"""
    
    def __init__(self):
        self.patterns = [
            (r'^[ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ]', 1, '로마숫자'),
            (r'^제\s*[0-9]+\s*장', 1, '제N장'),
            (r'^\d+\s+[가-힣A-Za-z]', 2, '숫자+공백'),
            (r'^\d+\.\s*[가-힣A-Za-z]', 2, '숫자.공백'),
            (r'^\d+\.\d+', 3, '숫자.숫자'),
            (r'^[□○●◆◇▪▫]', 3, '박스기호'),
            (r'^\(\d+\)', 3, '(숫자)'),
            (r'^\d+\)\s', 4, '숫자)'),
            (r'^[A-Z]\.\s', 4, '영대문자.'),
            (r'^[a-z]\.\s', 5, '영소문자.'),
        ]
    
    def detect_level_by_pattern(self, text):
        """패턴으로 레벨 추론"""
        text = text.strip()
        for pattern, level, name in self.patterns:
            if re.match(pattern, text):
                return level, name
        return None, None
    
    def detect_level_by_bbox(self, header):
        """BBox 크기로 레벨 추론"""
        if not header.prov:
            return None
        
        height = header.prov[0].bbox.t - header.prov[0].bbox.b
        
        if height > 25:
            return 1
        elif height > 18:
            return 2
        elif height > 12:
            return 3
        else:
            return 4
    
    def infer_hierarchy(self, section_headers):
        """섹션 헤더 목록에서 계층 추론"""
        results = []
        prev_level = 1
        
        for header in section_headers:
            text = header.text.strip()
            
            pattern_level, pattern_name = self.detect_level_by_pattern(text)
            bbox_level = self.detect_level_by_bbox(header)
            
            levels = []
            weights = []
            
            if pattern_level:
                levels.append(pattern_level)
                weights.append(3)
            
            if bbox_level:
                levels.append(bbox_level)
                weights.append(2)
            
            if levels:
                final_level = round(sum(l * w for l, w in zip(levels, weights)) / sum(weights))
            else:
                final_level = min(prev_level + 1, 5)
            
            confidence = 'high' if pattern_level else 'medium' if bbox_level else 'low'
            
            results.append({
                'text': text,
                'level': final_level,
                'confidence': confidence,
                'pattern': pattern_name,
                'page': header.prov[0].page_no if header.prov else None
            })
            
            prev_level = final_level
        
        return results

# =============================================================================
# Part 1: 문서 기본 정보
# =============================================================================

print("="*80)
print("Part 1: 문서 기본 정보 및 통계")
print("="*80)

print("\n=== new_dict 구조 ===")
for key, value in new_dict.items():
    if isinstance(value, dict):
        print(f"{key}: dict ({len(value)} items)")
    elif isinstance(value, list):
        print(f"{key}: list ({len(value)} items)")
    else:
        print(f"{key}: {type(value).__name__}")

print("\n=== 문서 메타데이터 ===")
print(f"파일명: {new_dict['name']}")
print(f"스키마: {new_dict['schema_name']}")
print(f"버전: {new_dict['version']}")
print(f"총 페이지: {len(new_dict['pages'])}")
print(f"총 텍스트 요소: {len(new_dict['texts'])}")
print(f"총 테이블: {len(new_dict['tables'])}")
print(f"총 이미지: {len(new_dict['pictures'])}")
print(f"총 그룹: {len(new_dict['groups'])}")

# 텍스트 라벨 분석
print("\n=== 텍스트 라벨 분포 ===")
labels = [str(text.label) for text in new_dict['texts'] if hasattr(text, 'label')]
label_counts = Counter(labels)

for label, count in label_counts.most_common():
    print(f"{label}: {count}")

# =============================================================================
# Part 2: 계층 구조 추론
# =============================================================================

print("\n" + "="*80)
print("Part 2: 계층 구조 추론")
print("="*80)

detector = HierarchyDetector()
section_headers = [t for t in new_dict['texts'] if hasattr(t, 'label') and str(t.label) == 'section_header']
hierarchy = detector.infer_hierarchy(section_headers)

print(f"\n총 섹션: {len(hierarchy)}")

# 계층 통계
level_counts = defaultdict(int)
for h in hierarchy:
    level_counts[h['level']] += 1

print("\n=== 계층 통계 ===")
for level in sorted(level_counts.keys()):
    print(f"Level {level}: {level_counts[level]}개")

print("\n=== 계층 구조 샘플 (첫 15개) ===")
for i, item in enumerate(hierarchy[:15]):
    indent = "  " * (item['level'] - 1)
    conf_icon = "✓" if item['confidence'] == 'high' else "~" if item['confidence'] == 'medium' else "?"
    print(f"{conf_icon} {indent}L{item['level']}: {item['text'][:60]}")

# =============================================================================
# Part 3: 계층적 문서 구조 추출
# =============================================================================

print("\n" + "="*80)
print("Part 3: 계층적 문서 구조")
print("="*80)

def extract_hierarchical_document(doc, hierarchy):
    """계층 정보를 포함한 문서 구조 추출"""
    
    hierarchy_map = {h['text']: h for h in hierarchy}
    
    document_structure = {
        'metadata': {
            'filename': doc.name,
            'pages': len(doc.pages),
            'schema': doc.schema_name,
            'version': doc.version
        },
        'content': []
    }
    
    current_path = []  # 현재 섹션 경로
    
    for child_ref in doc.body.children:
        if not hasattr(child_ref, 'cref'):
            continue
        
        child = resolve_ref(doc, child_ref.cref)
        if not child:
            continue
        
        child_type = type(child).__name__
        
        if child_type == 'SectionHeaderItem':
            text = child.text.strip()
            
            # 계층 정보 가져오기
            hier_info = hierarchy_map.get(text, {'level': 1, 'confidence': 'low'})
            
            section = {
                'type': 'section',
                'title': text,
                'page': child.prov[0].page_no if child.prov else None,
                'level': hier_info['level'],
                'confidence': hier_info['confidence'],
                'content': []
            }
            
            # 경로 업데이트
            current_path = [p for p in current_path if p['level'] < hier_info['level']]
            current_path.append(section)
            
            document_structure['content'].append(section)
        
        elif child_type == 'TextItem':
            if hasattr(child, 'label') and str(child.label) not in ['page_header', 'page_footer']:
                text_item = {
                    'type': 'text',
                    'content': child.text,
                    'page': child.prov[0].page_no if child.prov else None,
                    'label': str(child.label),
                    'section_path': ' > '.join([p['title'] for p in current_path])
                }
                
                if current_path:
                    current_path[-1]['content'].append(text_item)
                else:
                    document_structure['content'].append(text_item)
        
        elif child_type == 'ListGroup':
            list_group = {
                'type': 'list',
                'items': [],
                'enumerated': bool(getattr(child, 'first_item_is_enumerated', False)),
                'section_path': ' > '.join([p['title'] for p in current_path])
            }
            
            for list_child_ref in child.children:
                if hasattr(list_child_ref, 'cref'):
                    list_child = resolve_ref(doc, list_child_ref.cref)
                    if list_child and hasattr(list_child, 'text'):
                        list_group['items'].append({
                            'text': list_child.text,
                            'page': list_child.prov[0].page_no if hasattr(list_child, 'prov') and list_child.prov else None
                        })
            
            if current_path:
                current_path[-1]['content'].append(list_group)
            else:
                document_structure['content'].append(list_group)
        
        elif child_type == 'TableItem':
            table_item = {
                'type': 'table',
                'table_id': doc.tables.index(child),
                'page': child.prov[0].page_no if child.prov else None,
                'dimensions': {
                    'rows': child.data.num_rows if hasattr(child, 'data') else None,
                    'cols': child.data.num_cols if hasattr(child, 'data') else None
                },
                'markdown': safe_export_markdown(child),
                'section_path': ' > '.join([p['title'] for p in current_path])
            }
            
            if current_path:
                current_path[-1]['content'].append(table_item)
            else:
                document_structure['content'].append(table_item)
    
    return document_structure

hierarchical_doc = extract_hierarchical_document(doc, hierarchy)
sections_count = sum(1 for item in hierarchical_doc['content'] if item['type'] == 'section')

print(f"총 최상위 항목: {len(hierarchical_doc['content'])}")
print(f"섹션 수: {sections_count}")

# =============================================================================
# Part 4: RAG 검색 인덱스 (노이즈 제거 포함)
# =============================================================================

print("\n" + "="*80)
print("Part 4: RAG 검색 인덱스 생성 (노이즈 제거)")
print("="*80)

# 계층 경로 매핑
hierarchy_map = {h['text']: h for h in hierarchy}
current_path = []

search_index = []

# 텍스트 추가
for text in new_dict['texts']:
    if not hasattr(text, 'label'):
        continue
    
    label = str(text.label)
    
    # 섹션 헤더면 경로 업데이트
    if label == 'section_header':
        text_str = text.text.strip()
        if text_str in hierarchy_map:
            level = hierarchy_map[text_str]['level']
            current_path = [p for p in current_path if p['level'] < level]
            current_path.append({'level': level, 'text': text_str})
    
    # 검색 인덱스에 추가
    search_index.append({
        'content': text.text,
        'metadata': {
            'page': text.prov[0].page_no if text.prov else None,
            'type': 'text',
            'label': label,
            'section_path': ' > '.join([p['text'] for p in current_path]),
            'section_level_1': current_path[0]['text'] if len(current_path) > 0 else None,
            'section_level_2': current_path[1]['text'] if len(current_path) > 1 else None,
            'section_level_3': current_path[2]['text'] if len(current_path) > 2 else None,
        }
    })

# 테이블 행 추가
for i, table in enumerate(new_dict['tables']):
    page_no = table.prov[0].page_no if hasattr(table, 'prov') and table.prov else None
    
    if hasattr(table, 'data') and hasattr(table.data, 'grid'):
        for row_idx, row in enumerate(table.data.grid):
            row_text = ' | '.join([cell.text.strip() for cell in row])
            
            search_index.append({
                'content': row_text,
                'metadata': {
                    'page': page_no,
                    'type': 'table_row',
                    'table_id': i,
                    'row': row_idx,
                    'section_path': ' > '.join([p['text'] for p in current_path]),
                }
            })

print(f"원본 인덱스: {len(search_index)}개")

# 노이즈 제거
cleaner = SearchIndexCleaner()
cleaned_index = cleaner.clean_search_index(search_index)

print(f"정제 후 인덱스: {len(cleaned_index)}개")
print(f"제거율: {(len(search_index) - len(cleaned_index)) / len(search_index) * 100:.1f}%")

# =============================================================================
# Part 5: 파일 저장
# =============================================================================

print("\n" + "="*80)
print("Part 5: 파일 저장")
print("="*80)

# 1. 계층 구조 JSON
with open('document_hierarchical.json', 'w', encoding='utf-8') as f:
    json.dump(hierarchical_doc, f, ensure_ascii=False, indent=2)
print("✓ document_hierarchical.json 저장 완료")

# 2. 계층 정보 JSON
hierarchy_data = {
    'metadata': {
        'filename': new_dict['name'],
        'total_sections': len(hierarchy),
        'max_level': max(h['level'] for h in hierarchy) if hierarchy else 0
    },
    'hierarchy': hierarchy
}

with open('document_hierarchy.json', 'w', encoding='utf-8') as f:
    json.dump(hierarchy_data, f, ensure_ascii=False, indent=2)
print("✓ document_hierarchy.json 저장 완료")

# 3. 정제된 검색 인덱스 JSON
with open('search_index_cleaned.json', 'w', encoding='utf-8') as f:
    json.dump(cleaned_index, f, ensure_ascii=False, indent=2)
print("✓ search_index_cleaned.json 저장 완료")

# 4. 통합 메타데이터
full_metadata = {
    'document': {
        'filename': new_dict['name'],
        'total_pages': len(new_dict['pages']),
    },
    'statistics': {
        'texts': len(new_dict['texts']),
        'sections': len(hierarchy),
        'tables': len(new_dict['tables']),
        'images': len(new_dict['pictures']),
        'search_items_original': len(search_index),
        'search_items_cleaned': len(cleaned_index),
        'noise_removal_rate': f"{(len(search_index) - len(cleaned_index)) / len(search_index) * 100:.1f}%"
    },
    'text_labels': dict(label_counts),
    'hierarchy_levels': dict(level_counts)
}

with open('document_metadata.json', 'w', encoding='utf-8') as f:
    json.dump(full_metadata, f, ensure_ascii=False, indent=2)
print("✓ document_metadata.json 저장 완료")

print("\n" + "="*80)
print("완료! 생성된 파일:")
print("  - document_hierarchical.json (계층 구조)")
print("  - document_hierarchy.json (계층 정보)")
print("  - search_index_cleaned.json (정제된 RAG 인덱스)")
print("  - document_metadata.json (통합 메타데이터)")
print("="*80)


Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument i

Part 1: 문서 기본 정보 및 통계

=== new_dict 구조 ===
schema_name: str
version: str
name: str
origin: DocumentOrigin
furniture: GroupItem
body: GroupItem
groups: list (138 items)
texts: list (1546 items)
pictures: list (9 items)
tables: list (172 items)
key_value_items: list (0 items)
form_items: list (0 items)
pages: dict (131 items)

=== 문서 메타데이터 ===
파일명: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 
스키마: DoclingDocument
버전: 1.9.0
총 페이지: 131
총 텍스트 요소: 1546
총 테이블: 172
총 이미지: 9
총 그룹: 138

=== 텍스트 라벨 분포 ===
text: 757
list_item: 485
section_header: 150
page_footer: 129
page_header: 11
caption: 5
checkbox_unselected: 5
footnote: 4

Part 2: 계층 구조 추론

총 섹션: 150

=== 계층 통계 ===
Level 1: 9개
Level 2: 28개
Level 3: 79개
Level 4: 34개

=== 계층 구조 샘플 (첫 15개) ===
~     L3: 2024 2024 2024 2024 2024 2024 년 년 년 년 년 년 ｢ ｢ ｢ ｢ ｢ ｢ 벤처확인종합
~ L1: 제안요청서 제안요청서 제안요청서 제안요청서 제안요청서 제안요청서 제안요청서 제안요청서 제안요청서 제안요청서 
~     L3: 목 차
✓ L1: Ⅰ . 추진개요
✓   L2: 1 추진배경 및 방향
~       L4: 행정기관 및 공공기관 정보시스템

In [8]:
from pathlib import Path
import json
from datetime import datetime

def build_tree(hierarchy):
    """계층을 트리 구조로 변환"""
    tree = []
    stack = []  # (level, node)
    
    for item in hierarchy:
        node = {
            'text': item['text'],
            'level': item['level'],
            'page': item['page'],
            'children': []
        }
        
        # 스택에서 현재 레벨보다 같거나 낮은 레벨 제거
        while stack and stack[-1][0] >= item['level']:
            stack.pop()
        
        if stack:
            # 부모에 추가
            stack[-1][1]['children'].append(node)
        else:
            # 최상위 노드
            tree.append(node)
        
        stack.append((item['level'], node))
    
    return tree

def process_single_pdf(pdf_path, detector, cleaner, doc_converter):
    """단일 PDF 완전 처리"""
    
    try:
        # 1. PDF 파싱
        doc = doc_converter.convert(str(pdf_path)).document
        
        # 2. new_dict 생성
        new_dict = dict()
        for tuple_ in list(doc):
            new_dict[tuple_[0]] = tuple_[1]
        
        # 3. 섹션 헤더 추출 및 계층 추론
        section_headers = [
            t for t in new_dict['texts'] 
            if hasattr(t, 'label') and str(t.label) == 'section_header'
        ]
        hierarchy = detector.infer_hierarchy(section_headers)
        tree = build_tree(hierarchy)
        
        # 4. 계층적 문서 구조 추출
        hierarchy_map = {h['text']: h for h in hierarchy}
        current_path = []
        
        hierarchical_doc = {
            'metadata': {
                'filename': doc.name,
                'pages': len(new_dict['pages']),
                'schema': new_dict['schema_name'],
                'version': new_dict['version']
            },
            'content': []
        }
        
        # Body의 children 순회
        for child_ref in doc.body.children:
            if not hasattr(child_ref, 'cref'):
                continue
            
            child = resolve_ref(doc, child_ref.cref)
            if not child:
                continue
            
            child_type = type(child).__name__
            
            if child_type == 'SectionHeaderItem':
                text = child.text.strip()
                hier_info = hierarchy_map.get(text, {'level': 1, 'confidence': 'low'})
                
                section = {
                    'type': 'section',
                    'title': text,
                    'page': child.prov[0].page_no if child.prov else None,
                    'level': hier_info['level'],
                    'content': []
                }
                
                current_path = [p for p in current_path if p['level'] < hier_info['level']]
                current_path.append(section)
                hierarchical_doc['content'].append(section)
        
        # 5. 검색 인덱스 생성 (노이즈 제거 포함)
        search_index = []
        current_path_for_search = []
        
        for text in new_dict['texts']:
            if not hasattr(text, 'label'):
                continue
            
            label = str(text.label)
            
            # 섹션 경로 업데이트
            if label == 'section_header':
                text_str = text.text.strip()
                if text_str in hierarchy_map:
                    level = hierarchy_map[text_str]['level']
                    current_path_for_search = [p for p in current_path_for_search if p['level'] < level]
                    current_path_for_search.append({'level': level, 'text': text_str})
            
            search_index.append({
                'content': text.text,
                'metadata': {
                    'page': text.prov[0].page_no if text.prov else None,
                    'type': 'text',
                    'label': label,
                    'section_path': ' > '.join([p['text'] for p in current_path_for_search]),
                    'section_level_1': current_path_for_search[0]['text'] if len(current_path_for_search) > 0 else None,
                    'section_level_2': current_path_for_search[1]['text'] if len(current_path_for_search) > 1 else None,
                }
            })
        
        # 테이블 행 추가
        for i, table in enumerate(new_dict['tables']):
            page_no = table.prov[0].page_no if hasattr(table, 'prov') and table.prov else None
            
            if hasattr(table, 'data') and hasattr(table.data, 'grid'):
                for row_idx, row in enumerate(table.data.grid):
                    row_text = ' | '.join([cell.text.strip() for cell in row])
                    
                    search_index.append({
                        'content': row_text,
                        'metadata': {
                            'page': page_no,
                            'type': 'table_row',
                            'table_id': i,
                            'row': row_idx,
                            'section_path': ' > '.join([p['text'] for p in current_path_for_search]),
                        }
                    })
        
        # 노이즈 제거
        cleaned_index = cleaner.clean_search_index(search_index)
        
        # 6. 통계
        from collections import Counter
        labels = [str(t.label) for t in new_dict['texts'] if hasattr(t, 'label')]
        label_counts = Counter(labels)
        
        level_counts = {}
        for h in hierarchy:
            level_counts[h['level']] = level_counts.get(h['level'], 0) + 1
        
        # 7. 결과 반환
        return {
            'success': True,
            'filename': pdf_path.name,
            'data': {
                'metadata': {
                    'filename': doc.name,
                    'total_pages': len(new_dict['pages']),
                    'total_sections': len(hierarchy),
                    'max_level': max(h['level'] for h in hierarchy) if hierarchy else 0,
                    'total_texts': len(new_dict['texts']),
                    'total_tables': len(new_dict['tables']),
                    'total_images': len(new_dict['pictures']),
                },
                'statistics': {
                    'search_items_original': len(search_index),
                    'search_items_cleaned': len(cleaned_index),
                    'noise_removal_rate': f"{(len(search_index) - len(cleaned_index)) / len(search_index) * 100:.1f}%",
                    'text_labels': dict(label_counts),
                    'hierarchy_levels': level_counts
                },
                'hierarchy': hierarchy,
                'tree': tree,
                'hierarchical_structure': hierarchical_doc,
                'search_index_cleaned': cleaned_index
            }
        }
    
    except Exception as e:
        return {
            'success': False,
            'filename': pdf_path.name,
            'error': str(e)
        }

def process_all_pdfs(pdf_dir, output_dir, doc_converter):
    """
    100개 PDF 일괄 처리 (완전판)
    
    각 PDF마다 생성:
    - {name}_hierarchy.json (계층 정보)
    - {name}_hierarchical.json (계층 구조)
    - {name}_search_index.json (정제된 검색 인덱스)
    - {name}_metadata.json (메타데이터)
    
    전체 통합:
    - all_hierarchies.json (모든 PDF 계층 정보)
    - processing_report.json (처리 결과 리포트)
    """
    
    pdf_files = list(Path(pdf_dir).glob("*.pdf"))
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # 클래스 초기화
    detector = HierarchyDetector()
    cleaner = SearchIndexCleaner()
    
    # 결과 저장
    all_results = {}
    processing_log = {
        'start_time': datetime.now().isoformat(),
        'total_files': len(pdf_files),
        'success_count': 0,
        'failed_count': 0,
        'failed_files': [],
        'summary': {}
    }
    
    print(f"{'='*80}")
    print(f"PDF 일괄 처리 시작: {len(pdf_files)}개 파일")
    print(f"{'='*80}\n")
    
    for i, pdf_path in enumerate(pdf_files):
        print(f"[{i+1}/{len(pdf_files)}] 처리 중: {pdf_path.name}")
        
        # 처리
        result = process_single_pdf(pdf_path, detector, cleaner, doc_converter)
        
        if result['success']:
            doc_name = pdf_path.stem
            data = result['data']
            
            # 개별 파일 저장
            # 1. 계층 정보
            # with open(output_path / f"{doc_name}_hierarchy.json", 'w', encoding='utf-8') as f:
            #     json.dump({
            #         'metadata': data['metadata'],
            #         'hierarchy': data['hierarchy'],
            #         'tree': data['tree']
            #     }, f, ensure_ascii=False, indent=2)
            
            # # 2. 계층 구조
            # with open(output_path / f"{doc_name}_hierarchical.json", 'w', encoding='utf-8') as f:
            #     json.dump(data['hierarchical_structure'], f, ensure_ascii=False, indent=2)
            
            # # 3. 정제된 검색 인덱스
            # with open(output_path / f"{doc_name}_search_index.json", 'w', encoding='utf-8') as f:
            #     json.dump(data['search_index_cleaned'], f, ensure_ascii=False, indent=2)
            
            # # 4. 메타데이터
            # with open(output_path / f"{doc_name}_metadata.json", 'w', encoding='utf-8') as f:
            #     json.dump({
            #         'metadata': data['metadata'],
            #         'statistics': data['statistics']
            #     }, f, ensure_ascii=False, indent=2)
            
            # 전체 결과에 추가
            all_results[doc_name] = {
                'filename': result['filename'],
                'metadata': data['metadata'],
                'statistics': data['statistics']
            }
            
            processing_log['success_count'] += 1
            
            print(f"  ✓ 섹션: {data['metadata']['total_sections']}개")
            print(f"  ✓ 계층: {data['metadata']['max_level']}단계")
            print(f"  ✓ 검색 인덱스: {data['statistics']['search_items_original']} → {data['statistics']['search_items_cleaned']}개 ({data['statistics']['noise_removal_rate']} 제거)")
        
        else:
            processing_log['failed_count'] += 1
            processing_log['failed_files'].append({
                'filename': result['filename'],
                'error': result['error']
            })
            print(f"  ✗ 실패: {result['error']}")
    
    # 전체 통합 파일 저장
    with open(output_path / 'all_hierarchies.json', 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)
    
    # 처리 리포트
    processing_log['end_time'] = datetime.now().isoformat()
    processing_log['summary'] = {
        'total_sections': sum(r['metadata']['total_sections'] for r in all_results.values()),
        'total_pages': sum(r['metadata']['total_pages'] for r in all_results.values()),
        'avg_sections_per_doc': sum(r['metadata']['total_sections'] for r in all_results.values()) / len(all_results) if all_results else 0,
    }
    
    with open(output_path / 'processing_report.json', 'w', encoding='utf-8') as f:
        json.dump(processing_log, f, ensure_ascii=False, indent=2)
    
    # 결과 출력
    print(f"\n{'='*80}")
    print(f"처리 완료!")
    print(f"{'='*80}")
    print(f"성공: {processing_log['success_count']}개")
    print(f"실패: {processing_log['failed_count']}개")
    print(f"총 섹션: {processing_log['summary']['total_sections']}개")
    print(f"총 페이지: {processing_log['summary']['total_pages']}개")
    print(f"평균 섹션/문서: {processing_log['summary']['avg_sections_per_doc']:.1f}개")
    
    if processing_log['failed_files']:
        print(f"\n실패한 파일:")
        for failed in processing_log['failed_files']:
            print(f"  - {failed['filename']}: {failed['error']}")
    
    print(f"\n생성된 파일:")
    print(f"  - {output_path}/all_hierarchies.json (전체 계층 정보)")
    print(f"  - {output_path}/processing_report.json (처리 리포트)")
    print(f"  - {output_path}/{{name}}_*.json (개별 PDF 파일들)")
    print(f"{'='*80}\n")
    
    return all_results, processing_log

# =============================================================================
# 실행
# =============================================================================

# 예시:
all_results, report = process_all_pdfs(RAW_DIR, DATA_DIR, doc_converter)

# 또는 테스트용 (첫 3개만)
# test_files = list(Path(RAW_DIR).glob("*.pdf"))[:3]
# for pdf in test_files:
#     result = process_single_pdf(pdf, HierarchyDetector(), SearchIndexCleaner(), doc_converter)
#     print(f"{pdf.name}: {'✓' if result['success'] else '✗'}")


PDF 일괄 처리 시작: 100개 파일

[1/100] 처리 중: 사단법인 보험개발원_실손보험 청구 전산화 시스템 구축 사업.pdf
  ✓ 섹션: 135개
  ✓ 계층: 4단계
  ✓ 검색 인덱스: 2058 → 1308개 (36.4% 제거)
[2/100] 처리 중: 국가과학기술지식정보서비스_통합정보시스템 고도화 용역.pdf
  ✓ 섹션: 115개
  ✓ 계층: 4단계
  ✓ 검색 인덱스: 1872 → 931개 (50.3% 제거)
[3/100] 처리 중: 나노종합기술원_스마트 팹 서비스 활용체계 구축관련 설비온라인 시스.pdf
  ✓ 섹션: 77개
  ✓ 계층: 4단계
  ✓ 검색 인덱스: 2392 → 903개 (62.2% 제거)
[4/100] 처리 중: 부산관광공사_경영정보시스템 기능개선.pdf
  ✓ 섹션: 182개
  ✓ 계층: 4단계
  ✓ 검색 인덱스: 2527 → 1330개 (47.4% 제거)
[5/100] 처리 중: 축산물품질평가원_축산물이력관리시스템 개선(정보화 사업).pdf
  ✓ 섹션: 146개
  ✓ 계층: 4단계
  ✓ 검색 인덱스: 2235 → 1133개 (49.3% 제거)
[6/100] 처리 중: 재단법인스포츠윤리센터_스포츠윤리센터 LMS(학습지원시스템) 기능개선.pdf
  ✓ 섹션: 144개
  ✓ 계층: 4단계
  ✓ 검색 인덱스: 2340 → 1026개 (56.2% 제거)
[7/100] 처리 중: 한국농수산식품유통공사_농산물가격안정기금 정부예산회계연계

In [11]:
def process_all_pdfs(pdf_dir, output_dir, doc_converter, save_individual=False):
    """
    100개 PDF 일괄 처리 (통합 파일 중심)
    
    Args:
        pdf_dir: PDF 디렉토리
        output_dir: 출력 디렉토리
        doc_converter: Docling 변환기
        save_individual: 개별 파일 저장 여부 (기본: False)
    
    생성 파일:
        - all_search_index.json (통합 검색 인덱스) ⭐
        - all_hierarchies.json (통합 계층 정보)
        - all_metadata.json (통합 메타데이터)
        - processing_report.json (처리 리포트)
        - individual/*.json (옵션)
    """
    
    pdf_files = list(Path(pdf_dir).glob("*.pdf"))
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # 개별 파일 저장용 디렉토리
    if save_individual:
        individual_path = output_path / "individual"
        individual_path.mkdir(exist_ok=True)
    
    # 클래스 초기화
    detector = HierarchyDetector()
    cleaner = SearchIndexCleaner()
    
    # 통합 데이터 저장소
    all_search_items = []      # ⭐ 핵심: 모든 검색 항목
    all_hierarchies = {}       # 모든 계층 정보
    all_metadata = {}          # 모든 메타데이터
    
    processing_log = {
        'start_time': datetime.now().isoformat(),
        'total_files': len(pdf_files),
        'success_count': 0,
        'failed_count': 0,
        'failed_files': [],
        'statistics': {
            'total_search_items': 0,
            'total_sections': 0,
            'total_pages': 0
        }
    }
    
    print(f"{'='*80}")
    print(f"PDF 일괄 처리 시작: {len(pdf_files)}개 파일")
    print(f"{'='*80}\n")
    
    for i, pdf_path in enumerate(pdf_files):
        print(f"[{i+1}/{len(pdf_files)}] 처리 중: {pdf_path.name}")
        
        try:
            # 단일 PDF 처리
            result = process_single_pdf(pdf_path, detector, cleaner, doc_converter)
            
            if result['success']:
                doc_name = pdf_path.stem
                data = result['data']
                
                # 1. 검색 인덱스에 문서 정보 추가 ⭐
                for item in data['search_index_cleaned']:
                    # 메타데이터에 문서 정보 추가
                    item['metadata']['document'] = pdf_path.name
                    item['metadata']['document_id'] = doc_name
                    
                    all_search_items.append(item)
                
                # 2. 계층 정보 저장
                all_hierarchies[doc_name] = {
                    'filename': pdf_path.name,
                    'hierarchy': data['hierarchy'],
                    'tree': data['tree']
                }
                
                # 3. 메타데이터 저장
                all_metadata[doc_name] = {
                    'filename': pdf_path.name,
                    'metadata': data['metadata'],
                    'statistics': data['statistics']
                }
                
                # 4. 개별 파일 저장 (옵션)
                if save_individual:
                    with open(individual_path / f"{doc_name}_search_index.json", 'w', encoding='utf-8') as f:
                        json.dump(data['search_index_cleaned'], f, ensure_ascii=False, indent=2)
                    
                    with open(individual_path / f"{doc_name}_hierarchy.json", 'w', encoding='utf-8') as f:
                        json.dump(all_hierarchies[doc_name], f, ensure_ascii=False, indent=2)
                
                processing_log['success_count'] += 1
                processing_log['statistics']['total_search_items'] += len(data['search_index_cleaned'])
                processing_log['statistics']['total_sections'] += data['metadata']['total_sections']
                processing_log['statistics']['total_pages'] += data['metadata']['total_pages']
                
                print(f"  ✓ 검색 항목: {len(data['search_index_cleaned'])}개")
                print(f"  ✓ 섹션: {data['metadata']['total_sections']}개")
            
        except Exception as e:
            processing_log['failed_count'] += 1
            processing_log['failed_files'].append({
                'filename': pdf_path.name,
                'error': str(e)
            })
            print(f"  ✗ 실패: {e}")
    
    # =============================================================================
    # 통합 파일 저장 ⭐
    # =============================================================================
    
    print(f"\n{'='*80}")
    print("통합 파일 저장 중...")
    print(f"{'='*80}")
    
    # 1. 통합 검색 인덱스 (가장 중요!)
    with open(output_path / 'all_search_index.json', 'w', encoding='utf-8') as f:
        json.dump(all_search_items, f, ensure_ascii=False, indent=2)
    print(f"✓ all_search_index.json: {len(all_search_items)}개 항목")
    
    # 2. 통합 계층 정보
    with open(output_path / 'all_hierarchies.json', 'w', encoding='utf-8') as f:
        json.dump(all_hierarchies, f, ensure_ascii=False, indent=2)
    print(f"✓ all_hierarchies.json: {len(all_hierarchies)}개 문서")
    
    # 3. 통합 메타데이터
    with open(output_path / 'all_metadata.json', 'w', encoding='utf-8') as f:
        json.dump(all_metadata, f, ensure_ascii=False, indent=2)
    print(f"✓ all_metadata.json: {len(all_metadata)}개 문서")
    
    # 4. 처리 리포트
    processing_log['end_time'] = datetime.now().isoformat()
    with open(output_path / 'processing_report.json', 'w', encoding='utf-8') as f:
        json.dump(processing_log, f, ensure_ascii=False, indent=2)
    print(f"✓ processing_report.json")
    
    # 결과 출력
    print(f"\n{'='*80}")
    print(f"처리 완료!")
    print(f"{'='*80}")
    print(f"성공: {processing_log['success_count']}/{processing_log['total_files']}개")
    print(f"실패: {processing_log['failed_count']}개")
    print(f"")
    print(f"통합 검색 항목: {processing_log['statistics']['total_search_items']:,}개")
    print(f"총 섹션: {processing_log['statistics']['total_sections']:,}개")
    print(f"총 페이지: {processing_log['statistics']['total_pages']:,}개")
    print(f"평균 검색 항목/문서: {processing_log['statistics']['total_search_items'] / processing_log['success_count']:.0f}개")
    
    if processing_log['failed_files']:
        print(f"\n⚠️ 실패한 파일:")
        for failed in processing_log['failed_files']:
            print(f"  - {failed['filename']}: {failed['error']}")
    
    print(f"\n📁 생성된 파일:")
    print(f"  - {output_path}/all_search_index.json ⭐")
    print(f"  - {output_path}/all_hierarchies.json")
    print(f"  - {output_path}/all_metadata.json")
    print(f"  - {output_path}/processing_report.json")
    if save_individual:
        print(f"  - {output_path}/individual/*.json")
    print(f"{'='*80}\n")
    
    return {
        'search_items': all_search_items,
        'hierarchies': all_hierarchies,
        'metadata': all_metadata,
        'report': processing_log
    }

# =============================================================================
# 실행
# =============================================================================

# 권장: 통합 파일만 생성
results = process_all_pdfs(RAW_DIR, DATA_DIR, doc_converter, save_individual=False)

# 또는: 개별 파일도 저장 (디버깅용)
# results = process_all_pdfs(RAW_DIR, DATA_DIR, doc_converter, save_individual=True)

PDF 일괄 처리 시작: 100개 파일

[1/100] 처리 중: 사단법인 보험개발원_실손보험 청구 전산화 시스템 구축 사업.pdf
  ✓ 검색 항목: 1308개
  ✓ 섹션: 135개
[2/100] 처리 중: 국가과학기술지식정보서비스_통합정보시스템 고도화 용역.pdf
  ✓ 검색 항목: 931개
  ✓ 섹션: 115개
[3/100] 처리 중: 나노종합기술원_스마트 팹 서비스 활용체계 구축관련 설비온라인 시스.pdf
  ✓ 검색 항목: 903개
  ✓ 섹션: 77개
[4/100] 처리 중: 부산관광공사_경영정보시스템 기능개선.pdf
  ✓ 검색 항목: 1330개
  ✓ 섹션: 182개
[5/100] 처리 중: 축산물품질평가원_축산물이력관리시스템 개선(정보화 사업).pdf
  ✓ 검색 항목: 1133개
  ✓ 섹션: 146개
[6/100] 처리 중: 재단법인스포츠윤리센터_스포츠윤리센터 LMS(학습지원시스템) 기능개선.pdf
  ✓ 검색 항목: 1026개
  ✓ 섹션: 144개
[7/100] 처리 중: 한국농수산식품유통공사_농산물가격안정기금 정부예산회계연계시스템 .pdf
  ✓ 검색 항목: 839개
  ✓ 섹션: 108개
[8/100] 처리 중: 대한상공회의소_기업 재생에너지 지원센터 홈페이지 개편 및 시스템 고.pdf
  ✓ 검색 항목: 1004개
  ✓ 섹션: 109개
[9/100] 처리 중: 한ᄀ